___

# [ Geoinformatika ] 
Katedra aplikované geoinformatiky a kartografie, Universita Karlova

Lukas Brodsky lukas.brodsky@natur.cuni.cz


## GDAL-Python rastery


Tento notebook demonstruje zaklady prace s rastry v Python pomoci knihovny GDAL:  

* 1. Cteni rasteru do Numpy matice

* 2. Jednocucha mapova algebra 

* 3. Zapis rasteru na disk


### Priprava knihoven

In [ ]:
import os
import numpy as np
from osgeo import gdal, gdal_array

# Cesta k adresari s daty!!! 
# PATH = './'
PATH = '/mnt/home/k01/Downloads/geoinformatika-main/geodata'

In [ ]:
! ls $PATH

In [ ]:
! ls $PATH/raster

### Rasterova data

In [ ]:
raster_fn = os.path.join(PATH, 'raster', 'S2A_T33UVR_20180703T101029.tif')

### Cteni rasteru 

In [ ]:
# ovladac / driver 
driver = gdal.GetDriverByName('Gtiff')

In [ ]:
# otevreni 
ds = gdal.Open(raster_fn) 

In [ ]:
# raster dimensions 
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount

print('{} sloupcu, {} radek a {} kanalu'.format(cols, rows, bands))

In [ ]:
# geo meta
projection = ds.GetProjection()
geotransform = ds.GetGeoTransform()

In [ ]:
# geotransformacni metadata
originX = geotransform[0]
originY = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]

print('pocatek X: {}, pocatek Y: {} a velikost pixelu: {} m.'.format(originX, originY, pixelWidth))

In [ ]:
# nacteni vsech kanalu 
img = np.zeros((ds.RasterYSize, ds.RasterXSize, ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(ds.GetRasterBand(1).DataType))

for b in range(img.shape[2]):
    img[:, :, b] = ds.GetRasterBand(b + 1).ReadAsArray()

In [ ]:
img.shape

### Mapova algebra 

In [ ]:
RED = 1
NIR = 4
index = img[:,:,NIR] + img[:,:,RED]

In [ ]:
# ... jina analyza 

### Zapis noveho raster

In [ ]:
# priprava datove souboru, index.tif zmente na cokoliv jineho! 
jmeno_souboru = os.path.join(PATH, 'raster', 'index.tif')
vystup_ds = driver.Create(jmeno_souboru, ds.RasterXSize, ds.RasterYSize,  1, gdal.GDT_Int16)

In [ ]:
# zapsani matice 
vystupni_kanal = vystup_ds.GetRasterBand(1).WriteArray(index)

In [ ]:
# zapis geo metadat
vystup_ds.SetProjection(projection)
vystup_ds.SetGeoTransform(geotransform)

In [ ]:
# vystup_ds.GetRasterBand(1).SetNoDataValue(10000)

In [ ]:
# zapsani na disk 
vystup_ds = None

In [ ]:
# uklid
img = None
ds = None

In [ ]:
os.path.isfile(jmeno_souboru)